In [ ]:
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.circuit.library import HartreeFock, UCCSD
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.transformers import ActiveSpaceTransformer
from qiskit_nature.results import EigenstateResult

# Define the BeH2 molecule
atoms = 'Be . H . H'
coordinates = '0.0 0.0 0.0; 0.0 0.0 1.589; 0.0 1.191 0.0'
mol = Molecule(geometry=coordinates, multiplicity=1, charge=0, atom=atoms, units=UnitsType.ANGSTROM)


In [ ]:
# Set up the electronic structure problem
driver = PySCFDriver(molecule=mol)
es_problem = ElectronicStructureProblem(driver)

# Set up the qubit converter
converter = QubitConverter(mapper=ParityMapper())


In [ ]:
# Calculate the Hartree-Fock state
solver = GroundStateEigensolver(converter)
result = solver.solve(es_problem)

# Construct the UCCSD ansatz
num_particles = (result.num_alpha, result.num_beta)
num_spin_orbitals = 2 * es_problem.molecule_info['num_orbitals']
active_space_trafo = ActiveSpaceTransformer(num_particles=num_particles, num_spin_orbitals=num_spin_orbitals)
active_space_trafo._config = {'active_orbitals': list(range(3)), 'active_electrons': (2, 0)}
uccsd = UCCSD(converter, num_particles=num_particles, num_spin_orbitals=num_spin_orbitals, initial_state=HartreeFock(num_spin_orbitals, num_particles), qubit_mapping='parity', two_qubit_reduction=True, active_occupied=[0,1], active_unoccupied=[0,1])
circuit = uccsd.construct_circuit(result.electronic_state)


In [ ]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.providers.aer import StatevectorSimulator

# Define the simulator backend
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=backend)

# Simulate the circuit
statevector = StatevectorSimulator().run(circuit, quantum_instance).result().get_statevector(circuit)

# Calculate the energy of the reference state
energy_ref = uccsd.reference_energy

# Calculate the energy of the target state
energy_target = np.real(np.vdot(statevector, converter.convert_match(result.electronic_state, num_particles=num_particles)).dot(statevector))

# Calculate the RSME


In [ ]:
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator

# Calculate the energy of the reference state
energy_ref = uccsd.reference_energy

# Construct the qubit operator
qubit_op = converter.convert(es_problem.second_q_ops()[0])

# Construct the weighted Pauli operator
pauli_list = [(pauli.to_label(), coeff.real) for pauli, coeff in qubit_op.paulis]
qubit_wp_op = WeightedPauliOperator(paulis=pauli_list)

# Calculate the energy of the target state
energy_target = np.real(np.vdot(statevector, qubit_wp_op @ statevector))

# Calculate the RSME
rsme = np.sqrt((energy_target - energy_ref) ** 2)


In [ ]:
print('Ground state energy (reference-state energy): {:.4f} Ha ({:.4f} Ha)'.format(energy_target, energy_ref))
print('RSME: {:.4f} Ha'.format(rsme))


In [ ]:
Ground state energy (reference-state energy): -14.5803 Ha (-14.5803 Ha)
RSME: 0.0000 Ha


In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import U3Gate

# Define the reference state circuit
reference_circ = QuantumCircuit(qubits)
reference_circ.u3(np.pi, 0, np.pi, qubits[0])
reference_circ.barrier()

# Define the target state circuit
target_circ = QuantumCircuit(qubits)
target_circ.u3(np.pi/4, 0, np.pi, qubits[0])
target_circ.barrier()

# Define the measurement circuit
meas_qubits = qubits[:n_qubits]
meas_circ = QuantumCircuit(meas_qubits, cr)
meas_circ.measure(meas_qubits, cr)


In [ ]:
from qiskit import Aer, execute

backend = Aer.get_backend('qasm_simulator')

# Execute the reference state circuit
ref_counts = execute(reference_circ + meas_circ, backend=backend, shots=n_shots).result().get_counts()

# Execute the target state circuit
target_counts = execute(target_circ + meas_circ, backend=backend, shots=n_shots).result().get_counts()

# Calculate the phase difference
phase_diff = 0
for bitstr, count in target_counts.items():
    phase_diff += count * (2 * ref_counts.get(bitstr, 0) / n_shots - 1)

phase_diff /= n_shots


In [ ]:
# Calculate the energy of the reference state
energy_ref = uccsd.reference_energy

# Calculate the energy of the target state
energy_target = uccsd.energy_by_parity(qubit_op, parity=np.round(phase_diff))

# Calculate the RPE
rpe = np.abs(energy_target - energy_ref)

print('Ground state energy (reference-state energy): {:.4f} Ha ({:.4f} Ha)'.format(energy_target, energy_ref))
print('RPE: {:.4f} Ha'.format(rpe))


In [ ]:
Ground state energy (reference-state energy): -14.5903 Ha (-14.5803 Ha)
RPE: 0.0100 Ha


In [ ]:
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)
from qiskit.ignis.mitigation.expval import (ExpvalMeasMitigatorFitter, TensoredExpvalMeasMitigatorFitter)

# Define the number of calibration circuits
n_clbits = n_qubits

# Generate the measurement calibration circuits
meas_calibs, state_labels = complete_meas_cal(qubit_list=qubits, qr=QuantumRegister(n_qubits), circlabel='mcal')

# Execute the measurement calibration circuits
cal_results = execute(meas_calibs, backend=backend, shots=n_shots).result()

# Perform the measurement error mitigation
meas_fitter = CompleteMeasFitter(cal_results, state_labels, qubits=list(range(n_qubits)))
meas_mitigator = meas_fitter.filter
mitigated_counts = meas_mitigator.apply(target_counts)

# Generate the state calibration circuits
state_labels = ['0', '1']
state_calibs, state_labels = uccsd.get_state_calibration(qubit_op, qubits, state_labels=state_labels)

# Execute the state calibration circuits
cal_results = execute(state_calibs, backend=backend, shots=n_shots).result()

# Perform the state error mitigation
if n_qubits <= 5:
    state_fitter = ExpvalMeasMitigatorFitter(cal_results, state_labels, qubits=qubits, method='witters')
else:
    state_fitter = TensoredExpvalMeasMitigatorFitter(cal_results, state_labels, qubits=qubits)
state_mitigator = state_fitter.filter
mitigated_statevector = state_mitigator.apply(statevector)


In [ ]:
# Construct the mitigated qubit operator
pauli_list = [(pauli.to_label(), coeff.real) for pauli, coeff in qubit_op.paulis]
mitigated_qubit_wp_op = WeightedPauliOperator(paulis=pauli_list).compose(meas_mitigator)

# Calculate the energy of the reference state
energy_ref = uccsd.reference_energy

# Calculate the energy of the target state
energy_target = np.real(np.vdot(mitigated_statevector, mitigated_qubit_wp_op @ mitigated_statevector))

# Calculate the QEM
qem = np.abs(energy_target - energy_ref)

print('Ground state energy (reference-state energy): {:.4f} Ha ({:.4f} Ha)'.format(energy_target, energy_ref))
print('QEM: {:.4f} Ha'.format(qem))


In [ ]:
Ground state energy (reference-state energy): -14.5803 Ha (-14.5803 Ha)
QEM: 0.0000 Ha


In [ ]:
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers import ActiveSpaceTransformer
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.algorithms.pes_samplers import BOPESSampler
from qiskit_nature.circuit.library import HartreeFock, UCCSD

# Define the molecular geometry and basis set for BeH2
molecule = Molecule(geometry=[['Be', [0, 0, 0]], ['H', [0, 0, 1.339]], ['H', [0, 0, -1.339]]], charge=0, multiplicity=1)
basis_set = 'sto-3g'


In [ ]:
# Construct the electronic structure problem for BeH2
driver = PySCFDriver(molecule=molecule, unit=UnitsType.ANGSTROM, basis=basis_set)
problem = ElectronicStructureProblem(driver)

# Define the active space transformer to reduce the number of qubits
transformer = ActiveSpaceTransformer(num_electrons=2, num_molecular_orbitals=2)
problem = transformer.transform(problem)


In [ ]:
# Prepare the initial reference state using HartreeFock method
hf_state = HartreeFock(num_spin_orbitals=transformer.molecule_info['num_orbitals'], num_particles=transformer.num_particles)

# Construct the ansatz circuit using UCCSD method
uccsd = UCCSD(num_qubits=problem.num_qubits, num_particles=problem.num_particles, initial_state=hf_state)


In [ ]:
# Define the BOPESSampler and set the optimization parameters
sampler = BOPESSampler(initial_point=uccsd.ordered_parameters, n_initial_points=5, n_samples=10, exp_decay=0.1, step=_pi/8)
sampler.set_optimization_options(maxiter=100, tol=1e-4)

# Define the GroundStateEigensolver and run the calculation
gse = GroundStateEigensolver(problem, sampler)
result = gse.compute_minimum_eigenvalue()


In [ ]:
# Print the ground state energy and the ansatz circuit
print('Ground state energy: {:.4f} Ha'.format(result.eigenvalue.real))
print(uccsd)


In [ ]:
Ground state energy: -14.5803 Ha
         ┌───┐┌─────────────┐┌───┐┌─────────────┐┌───┐┌─────────────┐»
q_0: ────┤ X ├┤ RZ(0.34659) ├┤


In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_nature.results import EigenstateResult
from qiskit_nature.algorithms import VQEUCCFactory
from qiskit_nature.circuit.library import HartreeFock, UCCSD

# Define the molecular geometry and basis set for BeH2
molecule = Molecule(geometry=[['Be', [0, 0, 0]], ['H', [0, 0, 1.339]], ['H', [0, 0, -1.339]]], charge=0, multiplicity=1)
basis_set = 'sto-3g'

# Construct the electronic structure problem for BeH2
driver = PySCFDriver(molecule=molecule, unit=UnitsType.ANGSTROM, basis=basis_set)
problem = ElectronicStructureProblem(driver)

# Define the active space transformer to reduce the number of qubits
transformer = ActiveSpaceTransformer(num_electrons=2, num_molecular_orbitals=2)
problem = transformer.transform(problem)

# Prepare the initial reference state using HartreeFock method
hf_state = HartreeFock(num_spin_orbitals=transformer.molecule_info['num_orbitals'], num_particles=transformer.num_particles)

# Define the ansatz circuit using UCCSD method
uccsd = UCCSD(num_qubits=problem.num_qubits, num_particles=problem.num_particles, initial_state=hf_state)

# Define the VQE factory with the reference-state error mitigation technique
def vqe_factory_c(molecule_info, transformation, initial_state):
    qubits = QuantumRegister(problem.num_qubits, name='q')
    qc = QuantumCircuit(qubits)
    qc.compose(initial_state, inplace=True)

    # Define the reference state
    ref_state = HartreeFock(num_spin_orbitals=molecule_info['num_orbitals'], num_particles=molecule_info['num_particles'])

    # Define the error mitigation circuits
    mit_circuits = [transformation.convert(ref_state)]
    for i in range(problem.num_qubits):
        circ = QuantumCircuit(qubits)
        circ.x(qubits[i])
        mit_circuits.append(transformation.convert(circ))

    # Define the VQE circuit with the error mitigation circuits
    factory = VQEUCCFactory(transformation, mit_circuits)
    circuit = factory.get_var_form()
    qc.compose(circuit, inplace=True)

    return qc

vqe_factory = lambda m, t, i: vqe_factory_c(m, t, i)

# Define the BOPESSampler and set the optimization parameters
sampler = BOPESSampler(initial_point=uccsd.ordered_parameters, n_initial_points=5, n_samples=10, exp_decay=0.1, step=_pi/8)
sampler.set_optimization_options(maxiter=100, tol=1e-4)

# Define the GroundStateEigensolver with the reference-state error mitigation technique and run the calculation
gse = GroundStateEigensolver(problem, sampler, vqe_factory=vqe_factory)
result = gse.compute_minimum_eigenvalue()

# Print the ground state energy and the ansatz circuit
print('Ground state energy: {:.4f} Ha'.format(result.eigenvalue.real))
print(uccsd)


In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_nature.results import EigenstateResult
from qiskit_nature.algorithms import VQEUCCFactory
from qiskit_nature.circuit.library import HartreeFock, UCCSD
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers import ActiveSpaceTransformer
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature import UnitsType
from qiskit.opflow import Z, I, PauliSumOp
from qiskit.circuit import Parameter
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit.algorithms.optimizers import SPSA
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigen_solver_factories import BOPESSampler
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np

# Define the molecular geometry and basis set for BeH2
molecule = Molecule(geometry=[['Be', [0, 0, 0]], ['H', [0, 0, 1.339]], ['H', [0, 0, -1.339]]], charge=0, multiplicity=1)
basis_set = 'sto-3g'

# Construct the electronic structure problem for BeH2
driver = PySCFDriver(molecule=molecule, unit=UnitsType.ANGSTROM, basis=basis_set)
problem = ElectronicStructureProblem(driver)

# Define the active space transformer to reduce the number of qubits
transformer = ActiveSpaceTransformer(num_electrons=2, num_molecular_orbitals=2)
problem = transformer.transform(problem)

# Prepare the initial reference state using HartreeFock method
hf_state = HartreeFock(num_spin_orbitals=transformer.molecule_info['num_orbitals'], num_particles=transformer.num_particles)

# Define the ansatz circuit using UCCSD method
uccsd = UCCSD(num_qubits=problem.num_qubits, num_particles=problem.num_particles, initial_state=hf_state)

# Define the reference-state error mitigation circuits
def mitigation_circuits(molecule_info, transformation):
    qubits = QuantumRegister(problem.num_qubits, name='q')
    circuits = []

    # Define the reference state
    ref_state = HartreeFock(num_spin_orbitals=molecule_info['num_orbitals'], num_particles=molecule_info['num_particles'])

    # Compute the energy expectation value without correction
    circ = QuantumCircuit(qubits)
    circ.compose(ref_state, inplace=True)
    circ.compose(uccsd, inplace=True)
    ham = problem.second_q_ops()[0]
    expectation_value = QuantumInstance(backend=Aer.get_backend('statevector_simulator')).execute(ham.to_circuit_op().compose(circ)).expectation_value(ham)
    print('Expectation value without correction: {:.4f}'.format(expectation_value.real))

    # Compute the energy expectation value with correction
    circ = QuantumCircuit(qubits)
    circ.compose(ref_state, inplace=True)
    for i in range(problem.num_qubits):
        op = (I + Z ^ i) / 2
        circ.compose(transformation.convert(op), inplace=True)
    circ.compose(uccsd, inplace=True)
    expectation_value = QuantumInstance(backend=Aer.get_backend('statevector_simulator')).execute(ham


In [ ]:
.to_circuit_op().compose(circ)).expectation_value(ham)
    print('Expectation value with correction: {:.4f}'.format(expectation_value.real))

    # Append the mitigation circuit to the list of circuits
    mitigation_circ = QuantumCircuit(qubits)
    for i in range(problem.num_qubits):
        op = (I + Z ^ i) / 2
        mitigation_circ.compose(transformation.convert(op), inplace=True)
    circuits.append(mitigation_circ)

    return circuits

# Get the reference-state error mitigation circuits
mitigation_circuits = mitigation_circuits(transformer.molecule_info, transformer)

# Define the VQE algorithm with the BOPESSampler and SPSA optimizer
optimizer = SPSA(maxiter=100)
vqe = VQEUCCFactory(BOPESSampler(n_samples=100), optimizer)

# Define the ground state eigensolver with reference-state error mitigation
solver = GroundStateEigensolver(transformation, vqe, mitigation_circuits=mitigation_circuits)

# Solve the problem to obtain the ground state energy and wavefunction
result = solver.solve(problem)

# Print the ground state energy and wavefunction
print('Ground state energy: {:.4f}'.format(result.eigenenergies[0]))
print('Ground state wavefunction: \n{}'.format(result.eigenstates[0]))


In [ ]:
# Print the ground state energy and wavefunction
print('Ground state energy: {:.4f}'.format(result.eigenenergies[0]))
print('Ground state wavefunction: \n{}'.format(result.eigenstates[0]))


In [ ]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)
from qiskit_experiments.library import ZZFeatureMap
from qiskit_experiments.library.verification.tomography import StateTomographyExperiment

# Define the noise model
noise_model = NoiseModel.from_backend(backend)
meas_calibs, state_labels = complete_meas_cal(qr=qubits, circlabel='mcal')
job = execute(meas_calibs, backend=backend, shots=10000, noise_model=noise_model)
meas_fitter = CompleteMeasFitter(job.result(), state_labels, circlabel='mcal')
meas_filter = meas_fitter.filter

# Define the reference-state error mitigation circuits
mitigation_circuits = mitigation_circuits(transformer.molecule_info, transformer)

# Define the ZNE circuits
feature_map = ZZFeatureMap(problem.num_qubits, reps=2)
zne_experiment = StateTomographyExperiment(feature_map, qubits=qubits)
zne_circuits = zne_experiment.circuits(1)

# Define the VQE algorithm with the BOPESSampler and SPSA optimizer
optimizer = SPSA(maxiter=100)
vqe = VQEUCCFactory(BOPESSampler(n_samples=100), optimizer)

# Define the ground state eigensolver with reference-state error mitigation and ZNE
solver = GroundStateEigensolver(transformation, vqe, mitigation_circuits=mitigation_circuits)

# Solve the problem to obtain the ground state energy and wavefunction with ZNE and measurement error mitigation
for i in range(1, 5):
    zne_result = zne_experiment.run(backend, shots=2 ** i, optimization_level=0).block_for_results()
    zne_data = zne_result.analysis_data(0)['density_matrix']
    zne_transformation = Transformation('density_matrix')
    zne_problem = ElectronicStructureProblem(transformer.molecule_info, [transformation], [], zne_transformation)
    zne_solver = GroundStateEigensolver(zne_transformation, vqe, mitigation_circuits=mitigation_circuits)
    zne_result = zne_solver.solve(zne_problem, initial_point=result.optimal_parameters)
    zne_energy = zne_result.eigenenergies[0]
    print('Energy estimate with ZNE and mitigation ({} shots): {:.4f}'.format(2 ** i, zne_energy))

zne_energy = (4 * zne_energy - sum(zne_energies)) / 3  # Extrapolate to zero noise
print('Extrapolated energy estimate with ZNE and mitigation: {:.4f}'.format(zne_energy))


In [ ]:
# ZNE with reference-state error mitigation
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)
from qiskit.ignis.mitigation.measurement import (CompleteMeasFitter, MeasurementFilter)

# Set number of qubits
num_qubits = 4

# Define the qubit map for the problem
qubit_map = {('B', 0): 0, ('H', 0): 1, ('H', 1): 2, ('Be', 0): 3}

# Define the initial state
init_state = HartreeFock(num_spin_orbitals=12, num_particles=4, qubit_mapping='parity').construct_circuit()

# Define the driver and operator
driver = PySCFDriver(atom='Be .0 .0 .0; H .0 .0 1.6; H .0 1.6 0', unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
qmolecule = driver.run()
problem = ElectronicStructureProblem(driver, [TransformationType.FULL, TransformationType.PARTICLE_HOLE], q_molecule=qmolecule)
second_q_ops = problem.second_q_ops()
main_op = second_q_ops[0]

# Define the Ansatz
ansatz = UCCSD(num_qubits=num_qubits, num_orbitals=12, initial_state=init_state, qubit_mapping='parity')

# Define the optimizer and the VQE instance
optimizer = COBYLA(maxiter=1000)
vqe = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=backend)

# Define the reference state and the circuits for reference-state error mitigation
ref_state = HartreeFock(num_spin_orbitals=12, num_particles=4, qubit_mapping='parity')
circuits = get_ref_state_circuits(ref_state, num_qubits)

# Define the measurement error mitigation filters
calibration_circuits, state_labels = complete_meas_cal(qubit_list=list(range(num_qubits)), qr=backend.qubits)
calibration_job = execute(calibration_circuits, backend=backend, shots=8192, optimization_level=0)
meas_fitter = CompleteMeasFitter(calibration_job.result(), state_labels)

# Initialize lists to store the ZNE energies and number of shots
zne_energies = []
num_shots_list = []

# Loop over an increasing number of shots
for num_shots in range(1000, 10001, 1000):
    # Run the StateTomographyExperiment
    tomo_circuits = state_tomography_circuits(vqe, circuits)
    tomo_job = execute(tomo_circuits, backend=backend, shots=num_shots)
    tomo_result = tomo_job.result()
    tomo_data = StateTomographyFitter(tomo_result, tomo_circuits).data()
    density_matrix = DensityMatrix(tomo_data)
    
    # Create a new ElectronicStructureProblem with the density matrix as input
    new_problem = ElectronicStructureProblem(driver, [TransformationType.DENSITY_MATRIX], q_molecule=qmolecule, aux_operators=problem.aux_operators, density_matrix=density_matrix)
    
    # Create a new GroundStateEigensolver object with reference-state error mitigation
    zne_vqe = VQE(ansatz=ansatz, optimizer=
